<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tf-agents[reverb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 37.9 MB/s eta 0:00:00
   ━━━

In [2]:
import yfinance as yfi
import pandas as pd
import asyncio
#import tf_agents

In [3]:
def process_data(df):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    # Calculate the range and its standard deviation
    df['range'] = df['High'] - df['Low']
    stdev = df['range'].std()

    # Process data directly in one step
    processed_data = pd.DataFrame({
        'range': df['range'] / stdev,
        'close_percentage': (df['Close'] - df['Low']) / df['range'],
        'price_difference': (df['Close'] - df['Open']) / stdev
    })

    return processed_data


async def fetch_ticker_data(ticker: str, start_date: str, end_date: str, interval: str):
    try:
        data = yfi.download(ticker, start=start_date, end=end_date, interval=interval)
        proccessed_data = proccess_data(data)
        return proccessed_data

    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return None

async def fetch_all_tickers(ticker_list: list,start_date: str,end_date: str, interval: str, max_concurrency:int=50):
    tasks = [fetch_ticker_data(ticker,start_date=start_date,end_date=end_date,interval=interval) for ticker in ticker_list]
    return await asyncio.gather(*tasks[:max_concurrency])

In [4]:
start_date = '2023-01-01'
end_date = '2023-10-22'
interval='60m'

# List of forex tickers    # List of forex tickers
forex_tickers = ["EURUSD=X", "USDEUR=X","GBPUSD=X", 'USDGBP=x', "USDJPY=X","JPYUSD=X", "AUDUSD=X" , "USDAUD=X"]  # Example tickers

# Fetch data concurrently
results = await fetch_all_tickers(forex_tickers,start_date=start_date,end_date=end_date,interval=interval)

combined_df = pd.concat(results, axis=0, ignore_index=True)
combined_df.dropna(inplace=True)
target = combined_df['price_difference'][1:]

[*********************100%***********************]  1 of 1 completed


Error fetching data for EURUSD=X: Data must be 1-dimensional, got ndarray of shape (4996, 4997) instead


[*********************100%***********************]  1 of 1 completed


Error fetching data for USDEUR=X: Data must be 1-dimensional, got ndarray of shape (4996, 4997) instead


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Error fetching data for AUDUSD=X: Data must be 1-dimensional, got ndarray of shape (5001, 5002) instead


[*********************100%***********************]  1 of 1 completed


Error fetching data for USDAUD=X: Data must be 1-dimensional, got ndarray of shape (5001, 5002) instead


ValueError: All objects passed were None